In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install gdown
!pip install --upgrade gdown
!pip install patool
import patoolib

os.mkdir('unzipped_folder')

!gdown "https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download"

patoolib.extract_archive("/content/dataset.zip", outdir="/content/unzipped_folder")

ratings = pd.read_csv("/content/unzipped_folder/dataset/rating.csv")
animes = pd.read_csv("/content/unzipped_folder/dataset/anime.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 767.3 kB/s eta 0:00:00
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download
From (redirected): https://drive.google.com/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download&confirm=t&uuid=ae5a09dd-9aaa-4591-9c39-d65d1e52611b
To: /content/dataset.zip
100% 26.3M/26.3M [00:00<00:00, 71.2MB/s]
patool: Extracting /content/dataset.zip ...
patool: running /usr/bin/7z x -o/content/unzipped_folder -- /content/dataset.zip
patool: ... /c

In [3]:
tfidf = TfidfVectorizer(stop_words='english')

N = len(animes['name'])
id_dic = dict(zip(list(range(N)), (animes["anime_id"])))

def clean_data(x):

    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


features = ['name', 'genre', 'type', 'rating']

for feature in features:

    animes[feature] = animes[feature].apply(clean_data)


def create_soup(x):
  
    return ' ' + x['name'] + ' ' + x['genre'] + ' ' + x['type'] + ' ' + x['rating']


animes['soup'] = animes.apply(create_soup, axis=1)

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(animes['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(animes.index, index=animes['anime_id']).drop_duplicates()

def get_animes(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)
  all_animes = sel_sorted['anime_id'].tolist()
  top_animes = all_animes[0:20]

  return top_animes , all_animes


def get_recommendation_content_based(top_animes, all_animes, cosine_sim=cosine_sim2):

  rec_animes = []

  for i in top_animes:

    idx = indices[i]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]

    animes_indices = [i[0] for i in sim_scores]

    for j in animes_indices:

      anime_id = id_dic[j]

      if (anime_id not in rec_animes) and (anime_id not in all_animes):
        rec_animes.append(anime_id)

  return rec_animes


def get_similar_ids(top_animes, all_animes, cosine_sim=cosine_sim2):

  sim_ids = []

  for i in top_animes:

    idx = indices[i]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:41]

    animes_indices = [i[0] for i in sim_scores]

    for j in animes_indices:

      anime_id = id_dic[j]
      
      if (anime_id not in sim_ids) and (anime_id not in all_animes):
        sim_ids.append(anime_id)

  return sim_ids


animes_2 = pd.read_csv("/content/unzipped_folder/dataset/anime.csv")
ids_names = dict(zip(animes_2['anime_id'], animes_2['name'])) 

In [4]:
user_id = int(input("Enter user id: "))

top_animes, all_animes = get_animes(user_id)

similar_ids = get_similar_ids(top_animes, all_animes, cosine_sim2)
anime_ids = similar_ids + all_animes

new_ratings = ratings.loc[ratings['anime_id'].isin(anime_ids)]
new_ratings = new_ratings[new_ratings.rating != -1]

def get_top_rated_animes(user_id):

  select = new_ratings.loc[new_ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)
  rated_animes = sel_sorted['anime_id'].tolist()
  top_rated_animes = rated_animes[0:10]

  return top_rated_animes


def create_matrix(df):
	
	N = len(df['user_id'].unique())
	M = len(df['anime_id'].unique())
	
	user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
	anime_mapper = dict(zip(np.unique(df["anime_id"]), list(range(M))))
	
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
	anime_inv_mapper = dict(zip(list(range(M)), np.unique(df["anime_id"])))
	
	user_index = [user_mapper[i] for i in df['user_id']]
	anime_index = [anime_mapper[i] for i in df['anime_id']]

	X = csr_matrix((df["rating"], (anime_index, user_index)), shape=(M, N))
	
	return X, user_mapper, anime_mapper, user_inv_mapper, anime_inv_mapper


X, user_mapper, anime_mapper, user_inv_mapper, anime_inv_mapper = create_matrix(new_ratings)

kNN = NearestNeighbors(n_neighbors=10, algorithm="brute", metric='cosine')
kNN.fit(X)

def find_similar_animes(anime_id, k, show_distance=False):
      
    neighbour_ids = []

    anime_ind = anime_mapper[anime_id]
    anime_vec = X[anime_ind]
    anime_vec = anime_vec.reshape(1,-1)

    neighbour = kNN.kneighbors(anime_vec, return_distance=show_distance)

    for i in range(0,k):

        n = neighbour.item(i)
        neighbour_ids.append(anime_inv_mapper[n])

    neighbour_ids.pop(0)
    
    return neighbour_ids


act_users = new_ratings['user_id'].unique().tolist()

print("")
print("Recommended animes:")
print("")

if user_id in act_users:
  
  top_rated_animes = get_top_rated_animes(user_id)

  recommended_animes_comb = []
  
  for i in top_rated_animes:

      similar_animes = find_similar_animes(i, k=10)

      for j in similar_animes:
        
          if (j not in recommended_animes_comb) and (j not in all_animes):
            recommended_animes_comb.append(j)

  recommended_animes_comb = recommended_animes_comb[0:10]
  
  for i in recommended_animes_comb:
    
    print(ids_names[i])


else :

  top_animes_1 = all_animes[0:5]
  recommended_animes_content_based = get_recommendation_content_based(top_animes_1, all_animes, cosine_sim2)[0:10]

  for i in recommended_animes_content_based:

	  print(ids_names[i])  

Enter user id: 3

Recommended animes:

Pokemon: Mewtwo no Gyakushuu
Pokemon: Kesshoutou no Teiou Entei
Fullmetal Alchemist
Mirai Nikki (TV)
Clannad: After Story
Clannad
Kokoro Connect
Durarara!!
Kuroko no Basket: Tip Off
Shokugeki no Souma: Ni no Sara
